<a href="https://colab.research.google.com/github/hkolgur/UOH/blob/main/Amazon_Food_Reviews_bow_w2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

-Convert all the sentences into vectors to use the power of linear algebra
To Convert Text to Vectors, one of the techinique we can use is Bag Of Words.

Bag Of Words:

1) Construct  a  dictionary of all the unique words that are present in the reviews/documents.(eg:{ This, chips , very , tasty ..})

2) Construct a vector of d-dimension where d is the number of words present in corpus/dictionary that we created. Each word represents one dimention.

3) For each review/document ,construct vector such that each cell represents the number times a word is repeted in that particular review/documnet . Eg if This is repeated twice we put 2 . If Tasty is present once we put 1.. other clells we leave it as zeroes

4)Each vector (for each review is a sparse vector) because most of elements will be zerores.

5) For given 2 vectors compute the norm ||v1-v2|| 

Drawbacks of BOW:

If 2 reviews have same words with opposite menaing then the vectors sill have less distance between them and mislead to be simillar  
Eg The food is very tasy and affordable vs This is very bad and affordable

Boolean Bag Of Words:

 Instead of putting count of the occurances of words, we place 1 if word is present atleast from the corpus in the  review/document 0 otherwise
The difference between Vectors v1, v2 is the # of differing words (|V2-V1| = sqrt*(# of differing words) 

Text Pre-processing:

1) Filter stop words: Stop words do not convery much semantic meaning and hence can be ignored. Vectors will be meaningful and smaller if we remove stopwords
(sometimes removing stopwords is not good because words like not is also a part of stop words, which completely changes meaning of the review/document)

2) Convert all the text to lower case. Chips vs chips both coveys same meaning

3)Stemming: eg. tasty,taste,taseful are form same base/root word taste. Hence we instead of having 3 vectors we can represent them in common form .(Porter vs Snowball stemmer) 

4)Lemmitization:  How to break sentence into wrods .It is language dependent and context dependent. (Eg New York cannot be broken it should be one word. Each symbol is represented as unique word in Japnese/other languages) . Lemitizers group New York into one word .

5)Synoniyms like taste and delicious are same/simillar . But in BOW we consider them as different  as we dont consider semantic meaning of words . (Solution we use word2Vec) . 

Drawback of BOW is it does not take semantic meaning into consideration as we consider one dimension for every lemmitized word(uni-gram) 

One way to get around the problem is to use Bi-grams/Trigrams  (to get rid of lossing the opposite meaning by droping not as stop word) . This helps retain sequence information

Bi-Gram - 2 consequtive words are represented as one dimension
Tri-Gram - 3 consequtive words are represented as one dimension
n-Grams- n consequtive words are represented as one dimension

number of dimension in n-gram>Tri>Bigram>unigram

TF-IDF (Term Frequency -Inverse document Freq)

r,r2..rn n reviews with each review having w1,w2..wk words.
TF(Wi,rj) = # Times Wi occurs in rj / Total words in rj 
Hence TF of any word lies between 0 and 1( Prob of finding wi in rj) 

IDF-Inverse Document Frequency . 
If r1,r2..rN are N reviews and Dc is the Document corpus that contains all the words from all the reviews.
IDF(wi,Dc)= log( (Ni-Total number of documents)/ni -docs that contain wi)
ni is always <=Ni because documents that contain wi cannot be more than total docs

ni<=N => N/ni>=1 => log(N/ni)>=0 . Idf is > 0 for any word.
as ni increases log(N/ni) decreases . so if a word is more frequent then it has lower IDF value ( eg. The occurs in almost all documents)

Combine TF and IDF

For a particular word wj in a particular reveiw ri 
TF-IDF = Term Freq(word wj in review ri) * IDF(word wj in doc corpus Dc) 

TF-IDF gives more importance to the rare words. 

Drawback it still do not take semantic meaning into consideration. Eg. Tasty and delicious are same but it doesnt consider.

why use log? in TF-IDF - no strong theory around it but used as a hack
The frequency of words in english language follows a power law distribution(almsot) Hence taking we can conver it to Gausian dist with Box cox transformation.(core ideas of box cox is to take log or r.v) 

Other reason is if we use N/ni -> The gets almsot 1 and rare word that is one in 1000 gets 1000/1 = 1000 . The gap is very large instead if we take log it will be 6.9 instead of 1000 . This help to reduce the impact of IDF value which will dominate otherwise and TF will not have essence in rare words



Word2Vec:

BOW and TF-IDF do not take semantic meaning into consideration.
Takes a word and converts into a vector. if 2 words are given it gives 2 vectors in such a way the relationship between words are preserved.
Eg. Man is opposite gender of Women. King is opposite gender of Queen
Now V-man - V-women is parallel to V-king -V-QUeen

Core Intuition of word2Vec is that vector for a word w3 takes into consideration of the words in its Neighbourhood like w1,w2, w4,w5. If Neighbourhood of wi is simillar to word wj then the vecotros of these 2 words vi and vj will be very simillar  

Give large text documents (eg 500k reviews) to w2Vec and train. It then generates vectors for each word in review to predict.

Average WOrd2Vec
w2vec converts a word to vector but bow to convert a sentence into a vector? One way is to take average w2vec of each word in the review.
ie sum all the vectros of each word and divide by the number of words in that reviw to get average of the vectors

TF-IDF weighted word2vec:

for Review r1 first compute the TF-IDF for each word say t1, t2,t3... , Also compute the word2Vec of each word w2v(w1),w2v(w2),...
Now mulitply each (t1*w2v(w1) + t2*w2v(t2) ..)/(t1 + t2...)  . Here if t1=t2=..=1 ie all are 1 then it is equvilant to average word2vec



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
import re
import numpy as np

In [4]:
con=sqlite3.connect('/content/drive/MyDrive/UOH/AmazonReviews/database.sqlite')


In [5]:
filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 5000""",con) 

In [6]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [7]:
print(sum(filtered_data['Score']<3))# count of Negative reviews
print(sum(filtered_data['Score']>3)) # count of Positive reviews

813
4187


In [8]:
#Function to determine if the review is positive or negative based on the Score.
def positive_Negative(x):
  if x<3:
    return 0
  return 1

#Create a new column Sentiment which is derived based on the values of Score column
filtered_data['Sentiment']=filtered_data.apply(lambda row: positive_Negative(row['Score']),axis=1)

In [9]:
filtered_data[['Score','Sentiment']]

,Score,Sentiment
0,5,1
1,1,0
2,4,1
3,2,0
4,5,1
...,...,...
4995,2,0
4996,4,1
4997,5,1
4998,2,0


In [10]:
#Drop the Score column
filtered_data.drop(columns='Score',inplace=True)

In [11]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Sentiment
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1


In [12]:
filtered_data['Text']

0       I have bought several of the Vitality canned d...
1       Product arrived labeled as Jumbo Salted Peanut...
2       This is a confection that has been around a fe...
3       If you are looking for the secret ingredient i...
4       Great taffy at a great price.  There was a wid...
                              ...                        
4995    My baby didn't seem into these dinners, so I t...
4996    This is great!  Organic baby food options - de...
4997    My little guy loves to try new foods..so this ...
4998    We ordered the Earth's best 2nd dinner variety...
4999    My baby loves this food.  At whole foods they ...
Name: Text, Length: 5000, dtype: object

In [13]:
# select specific columns
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", con)

In [14]:
#Check the  maximum number of reviews given by any person 
display['COUNT(*)'].max()

448

In [15]:
#Lets check the userid of the person who gave maximum number of reviews
display[display['COUNT(*)']==448]

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
57359,A3OXHLG6DIBRW8,B005K4Q68Q,"C. F. Hill ""CFH""",1321401600,5,These Grove Square Hot Cocoa flavors are by fa...,448


In [16]:
filtered_data[filtered_data['UserId']=='A3OXHLG6DIBRW8']

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Sentiment
338,370,B002O3VHXU,A3OXHLG6DIBRW8,"C. F. Hill ""CFH""",1,1,1282176000,Very Smooth Coffee - Highly Recommended,"Green Mountain ""Nantucket Blend"" K-Cups make a...",1
753,814,B004ET7MG8,A3OXHLG6DIBRW8,"C. F. Hill ""CFH""",3,4,1272240000,Odd Fake Flavor - Not Recommended,"Trident ""Strawberry Twist"" sugarless gum is ve...",0
3034,3307,B005K4Q1VI,A3OXHLG6DIBRW8,"C. F. Hill ""CFH""",24,25,1321401600,Really Good Hot Cocoa - Highly Recommended,These Grove Square Hot Cocoa flavors are by fa...,1
3139,3417,B005K4Q1VI,A3OXHLG6DIBRW8,"C. F. Hill ""CFH""",5,5,1321401600,Really Good Hot Cocoa - Highly Recommended,These Grove Square Hot Cocoa flavors are by fa...,1
3613,3927,B000VSDFRG,A3OXHLG6DIBRW8,"C. F. Hill ""CFH""",2,2,1237161600,Great Diabetic Friendly Candy - Highly Recomme...,"Hershey ""Sugar Free Caramel Filled Chocolates""...",1




```
# This is formatted as code
```

# Exploratory Data Analysis

#### Lets review the products reviewed by user AR5J8UI46CURR

In [17]:
display=pd.read_sql_query("""select * from Reviews where Userid=='AR5J8UI46CURR'""",con)

In [18]:
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


#### Observation: For this particular user we observe that the score is same for all producta and even the time stamp of review is same. when the User Rated one item, the review might have been mapped to multiple items in that particular brand/group

#### Lets Sort the data and remove duplicates . we can define the critera of a duplicate if it has same "UserId","ProfileName","Time","Text

In [19]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values(by='ProductId',kind='quicksort',ascending=True,axis=0,na_position='last',inplace=False)

In [20]:
sorted_data.shape

(5000, 10)

In [21]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},inplace=False,keep='first')

In [22]:
final.shape 

(4986, 10)

In [23]:
#Checking to see how much % of data still remains
(final.shape[0]/sorted_data.shape[0])*100

99.72

In [24]:
#select the rows that have helpfulness Numerator less than or= denominator ( if > it is an error)

final=final[final['HelpfulnessNumerator']<=final['HelpfulnessDenominator']]


In [25]:
#lets see the number of entries left
print(final.shape)
#lets see the number of entries with positive and Negative sentiment
print(final['Sentiment'].value_counts())

(4986, 10)
1    4178
0     808
Name: Sentiment, dtype: int64


# Text Pre-Processing 

In [26]:
# Lets view first review 
final['Text'].values[0]

'Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.'

#### Observation: Above we see there are lot of html tags present as part of the revoew . we have to clean it

In [27]:
# printing some random reviews
sent_0 = final['Text'].values[0]
print(sent_0)
print("="*50)

sent_1000 = final['Text'].values[1000]
print(sent_1000)
print("="*50)

sent_1500 = final['Text'].values[1500]
print(sent_1500)
print("="*50)

sent_4900 = final['Text'].values[4900]
print(sent_4900)
print("="*50)

Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.
I recently tried this flavor/brand and was surprised at how delicious these chips are.  The best thing was that there were a lot of "brown" chips in the bsg (my favorite), so I bought some more through amazon and shared with family and friends.  I am a little disappointed that there are not, so far, very many brown chips in these bags, but the flavor is still very good.  I like them better than the yogurt and green onion flavor because they do not seem to be as salty, and the onion flavor is better.  If you haven't eaten Kettle chips before, I recommend that you try a bag before buying bulk.  They are thicker and crunchier than Lays but just as fresh out of the bag.
Wow.  So far, two two-star reviews.  One obviously had no 

In [28]:
# remove urls from text python: https://stackoverflow.com/a/40823105/4084039
sent_0 = re.sub(r"http\S+", "", sent_0)
sent_1000 = re.sub(r"http\S+", "", sent_1000)
sent_1500= re.sub(r"http\S+", "", sent_1500)
sent_4900 = re.sub(r"http\S+", "", sent_4900)

print(sent_0)

Why is this $[...] when the same product is available for $[...] here?<br /> /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [29]:
soup=BeautifulSoup(sent_0,'lxml')
text=soup.get_text()
print(text)
print("="*50)

soup=BeautifulSoup(sent_1000,'lxml')
text=soup.get_text()
print(text)
print("="*50)

soup=BeautifulSoup(sent_1500,'lxml')
text=soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_4900, 'lxml')
text = soup.get_text()
print(text)


Why is this $[...] when the same product is available for $[...] here? />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.
I recently tried this flavor/brand and was surprised at how delicious these chips are.  The best thing was that there were a lot of "brown" chips in the bsg (my favorite), so I bought some more through amazon and shared with family and friends.  I am a little disappointed that there are not, so far, very many brown chips in these bags, but the flavor is still very good.  I like them better than the yogurt and green onion flavor because they do not seem to be as salty, and the onion flavor is better.  If you haven't eaten Kettle chips before, I recommend that you try a bag before buying bulk.  They are thicker and crunchier than Lays but just as fresh out of the bag.
Wow.  So far, two two-star reviews.  One obviously had no idea what they were ordering; the other wants crispy cookies.  Hey, I'm sorry; b

In [30]:
# Method to remove abbrevations and substitute with full words
#Eg: change can't to can not .

def substitutions(phrase):
  # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [31]:
sent_1500=substitutions(sent_1500)
print(sent_1500)

Wow.  So far, two two-star reviews.  One obviously had no idea what they were ordering; the other wants crispy cookies.  Hey, I am sorry; but these reviews do nobody any good beyond reminding us to look  before ordering.<br /><br />These are chocolate-oatmeal cookies.  If you do not like that combination, do not order this type of cookie.  I find the combo quite nice, really.  The oatmeal sort of "calms" the rich chocolate flavor and gives the cookie sort of a coconut-type consistency.  Now let is also remember that tastes differ; so, I have given my opinion.<br /><br />Then, these are soft, chewy cookies -- as advertised.  They are not "crispy" cookies, or the blurb would say "crispy," rather than "chewy."  I happen to like raw cookie dough; however, I do not see where these taste like raw cookie dough.  Both are soft, however, so is this the confusion?  And, yes, they stick together.  Soft cookies tend to do that.  They are not individually wrapped, which would add to the cost.  Oh y

In [32]:
#remove words with numbers python: https://stackoverflow.com/a/18082370/4084039
sent_0 = re.sub("\S*\d\S*", "", sent_0).strip()
print(sent_0)

Why is this $[...] when the same product is available for $[...] here?<br /> /><br />The Victor  and  traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [33]:
#remove spacial character: https://stackoverflow.com/a/5843547/4084039
sent_1500 = re.sub('[^A-Za-z0-9]+', ' ', sent_1500)
print(sent_1500)

Wow So far two two star reviews One obviously had no idea what they were ordering the other wants crispy cookies Hey I am sorry but these reviews do nobody any good beyond reminding us to look before ordering br br These are chocolate oatmeal cookies If you do not like that combination do not order this type of cookie I find the combo quite nice really The oatmeal sort of calms the rich chocolate flavor and gives the cookie sort of a coconut type consistency Now let is also remember that tastes differ so I have given my opinion br br Then these are soft chewy cookies as advertised They are not crispy cookies or the blurb would say crispy rather than chewy I happen to like raw cookie dough however I do not see where these taste like raw cookie dough Both are soft however so is this the confusion And yes they stick together Soft cookies tend to do that They are not individually wrapped which would add to the cost Oh yeah chocolate chip cookies tend to be somewhat sweet br br So if you wa

In [34]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [35]:
#split the sentence with space , convert each word to lower case ,remove stop words and then combine the sentence
' '.join(e.lower() for e in sent_1500.split() if e.lower() not in stopwords)


'wow far two two star reviews one obviously no idea ordering wants crispy cookies hey sorry reviews nobody good beyond reminding us look ordering chocolate oatmeal cookies not like combination not order type cookie find combo quite nice really oatmeal sort calms rich chocolate flavor gives cookie sort coconut type consistency let also remember tastes differ given opinion soft chewy cookies advertised not crispy cookies blurb would say crispy rather chewy happen like raw cookie dough however not see taste like raw cookie dough soft however confusion yes stick together soft cookies tend not individually wrapped would add cost oh yeah chocolate chip cookies tend somewhat sweet want something hard crisp suggest nabiso ginger snaps want cookie soft chewy tastes like combination chocolate oatmeal give try place second order'

In [36]:
# Combining all the above  

from tqdm import tqdm
# tqdm is for printing the status bar

preprocessed_reviews = []

for sentance in tqdm(final['Text'].values):
  sentance=re.sub('http\S+','',sentance)
  sentance = BeautifulSoup(sentance, 'lxml').get_text()
  sentence=substitutions(sentance)
  sentance = re.sub("\S*\d\S*", "", sentance).strip()
  sentance = re.sub('[^A-Za-z]+', ' ', sentance)

  sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
  preprocessed_reviews.append(sentance.strip())


100%|██████████| 4986/4986 [00:01<00:00, 2762.04it/s]


In [37]:
preprocessed_reviews[0:10]

['product available victor traps unreal course total fly genocide pretty stinky right nearby',
 'used victor fly bait seasons beat great product',
 'received shipment could hardly wait try product love slickers call instead stickers removed easily daughter designed signs printed reverse use car windows printed beautifully print shop program going lot fun product windows everywhere surfaces like tv screens computer monitors',
 'really good idea final product outstanding use decals car window everybody asks bought decals made two thumbs',
 'glad cocker standard poodle puppy loves stuff trust brand superior nutrition compare labels previous feed pedigree mostly corn little dude healthy happy high energy glossy coat also superior nutrition produces smaller compact stools',
 'using food months find excellent fact two dogs coton de tulear standard poodle puppy love food thriving coats excellent condition overall structure perfect good tasting dog good good deal owner around best food ever us

# Featurization 

# BOW

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

#BoW
count_vect = CountVectorizer() 
count_vect.fit(preprocessed_reviews)
print("some feature names ", count_vect.get_feature_names()[:10])
print('='*50)

final_counts = count_vect.transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_counts))
print("the shape of out text BOW vectorizer ",final_counts.get_shape())
print("the number of unique words ", final_counts.get_shape()[1])

some feature names  ['aa', 'aahhhs', 'aback', 'abandon', 'abates', 'abbott', 'abby', 'abdominal', 'abiding', 'ability']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (4986, 12974)
the number of unique words  12974


In [39]:
# size of the vocabulary 
len(count_vect.vocabulary_)

12974

# Bi-Grams and n-grams

In [40]:
count_vect=CountVectorizer(preprocessed_reviews,ngram_range=(1,2),min_df=10, max_features=5000)
count_vect.fit(preprocessed_reviews)  
# size of the vocabulary 
print("Vocabulary Length:",len(count_vect.vocabulary_))

final_bigram_counts=count_vect.transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_bigram_counts))
print("the shape of out text BOW vectorizer ",final_bigram_counts.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_bigram_counts.get_shape()[1])

Vocabulary Length: 3029
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (4986, 3029)
the number of unique words including both unigrams and bigrams  3029


In [41]:
# Tri-Grams
count_vect=CountVectorizer(preprocessed_reviews,ngram_range=(1,3),min_df=10, max_features=5000)
count_vect.fit(preprocessed_reviews)  

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8',
                input=['product available victor traps unreal course total fly '
                       'genocide pretty stinky right nearby',
                       'used victor fly bait seasons beat great product',
                       'received shipment could hardly wait try product love '
                       'slickers call instead stickers removed easily daughter '
                       'designed...
                       'like bean skins get paste put ice cream fill pasteries '
                       'stuff mochi cakes',
                       'good beans find grocery stores live ordered retailer '
                       'no problems',
                       'not good yummy smell like cloves cooking taste little '
                       'sweet', ...],
                lowercase=True, max_df=1.0, max_features=5000, min_df=10,
                ngra

In [42]:
print("Vocabulary Length:",count_vect.vocabulary_)

Vocabulary Length: {'product': 2106, 'available': 135, 'course': 592, 'total': 2784, 'pretty': 2080, 'right': 2267, 'nearby': 1730, 'used': 2864, 'beat': 188, 'great': 1151, 'great product': 1163, 'received': 2200, 'shipment': 2400, 'could': 579, 'hardly': 1225, 'wait': 2907, 'try': 2816, 'love': 1552, 'call': 348, 'instead': 1351, 'easily': 775, 'daughter': 644, 'designed': 685, 'use': 2859, 'car': 367, 'windows': 2982, 'shop': 2406, 'program': 2117, 'going': 1099, 'lot': 1550, 'fun': 1043, 'everywhere': 849, 'like': 1471, 'computer': 532, 'wait try': 2908, 'really': 2180, 'good': 1103, 'idea': 1319, 'outstanding': 1917, 'bought': 276, 'made': 1585, 'two': 2832, 'really good': 2183, 'good idea': 1113, 'glad': 1082, 'cocker': 484, 'standard': 2531, 'puppy': 2129, 'loves': 1570, 'stuff': 2588, 'trust': 2815, 'brand': 284, 'superior': 2612, 'nutrition': 1830, 'compare': 523, 'labels': 1423, 'previous': 2084, 'feed': 908, 'mostly': 1696, 'corn': 573, 'little': 1510, 'healthy': 1238, 'happ

# TF-IDF


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
tf_idf_vect.fit(preprocessed_reviews)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

final_tf_idf = tf_idf_vect.transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_tf_idf))
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

some sample features(unique words in the corpus) ['ability', 'able', 'able find', 'able get', 'absolute', 'absolutely', 'absolutely delicious', 'absolutely love', 'absolutely no', 'according']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (4986, 3029)
the number of unique words including both unigrams and bigrams  3029


Info: Above we see the type of tf-idf-vector after fit_transform is a sparse matrix. Each row of the matrix is a particular review. Each column in row corresponds to the tf-idf value of particular feature /word . It is sparse because all the words form corpus will not be present in each review.

In [44]:
# print the features names form 1000 to 1010 
print("features names form 1000 to 1010",tf_idf_vect.get_feature_names()[1000:1010]) 
features=tf_idf_vect.get_feature_names()
print("Total Features size:",len(features))

features names form 1000 to 1010 ['fragile', 'free', 'free bisquick', 'free diet', 'free foods', 'free product', 'free products', 'free shipping', 'freeze', 'freezer']
Total Features size: 3029


In [45]:
# Get a vector corresponding to a review eg r3

final_tf_idf[3,:].toarray() # row 3 all the columns 


array([[0., 0., 0., ..., 0., 0., 0.]])

In [46]:
# Get top n features related to a sentence/review 

def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

top_tf_idf=top_tfidf_feats(final_tf_idf[3,:].toarray()[0],features,25)

In [47]:
print(preprocessed_reviews[3])
top_tf_idf

really good idea final product outstanding use decals car window everybody asks bought decals made two thumbs


,feature,tfidf
0,good idea,0.434395
1,outstanding,0.414054
2,car,0.404419
3,idea,0.326540
4,really good,0.307497
5,two,0.234951
6,made,0.220373
7,bought,0.219595
8,use,0.198740
9,really,0.186046


In [69]:
#values of the finial_tf_idf shows it has the tfids values for few words and 0 for words that are not present in review
final_tf_idf[3,:].toarray().reshape(233,13).tolist()

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [70]:
#sample to understand . As displayed above we are sorting the arguments that have top values of tfid's.
#then we take the particular tf-idf values and find corresponding feature 
topn_ids = np.argsort(final_tf_idf[3,:].toarray()[0])[::-1][:25]
print("Argument positions of top 25 ids",topn_ids)
print("Feature is : ",features[1100])
print("TF-IDF value:",final_tf_idf[3,:].toarray()[0][1100])

Argument positions of top 25 ids [1100 1891  363 1299 2156 2797 1560  273 2824 2153 2079 1090 1004 1010
  995 1009 1008 1007 1006 1005  996  997 1003 1012 1002]
Feature is :  good idea
TF-IDF value: 0.4343950081032161


# Word2Vec

In [71]:
print("Sample Review before split",preprocessed_reviews[1])
print("Sample Review after split",preprocessed_reviews[1].split())

Sample Review before split used victor fly bait seasons beat great product
Sample Review after split ['used', 'victor', 'fly', 'bait', 'seasons', 'beat', 'great', 'product']


In [73]:
# Train your own Word2Vec model using your own text corpus
list_of_sentance=[]
for sentance in preprocessed_reviews:
    list_of_sentance.append(sentance.split())

In [74]:
from gensim.models import Word2Vec

w2v_model=Word2Vec(list_of_sentance,min_count=5,size=50, workers=4) #Ignores all words with total frequency lower than 5.
# size is the dimension of the vector that the word is converted into. 
# Words below the min_count frequency are dropped before training occurs. So, the relevant context window is the word-distance 
# among surviving words.
# This de facto shrinking of contexts is usually a good thing: the infrequent words don't have enough varied examples to obtain 
# good vectors for themselves. Further, while individually each infrequent word is rare, in total there are lots of them, so these
#  doomed-to-poor-vector rare-words intrude on most other words' training, serving as a sort of noise that makes those word-vectors
#   worse too.

# (Similarly, when using the sample parameter to down-sample frequent words, the frequent words are randomly dropped – which also 
# serves to essentially "shrink" the distances between surviving words, and often improves overall vector quality.)

print(w2v_model.wv.most_similar('great'))
print('='*50)
print(w2v_model.wv.most_similar('worst'))

[('alternative', 0.9959698915481567), ('especially', 0.9957107305526733), ('tasty', 0.995488166809082), ('snack', 0.9954031109809875), ('excellent', 0.9949555397033691), ('though', 0.9948632717132568), ('anything', 0.9946957230567932), ('looking', 0.994658350944519), ('want', 0.9945265054702759), ('satisfying', 0.9945114254951477)]
[('popcorn', 0.9993431568145752), ('clam', 0.9993214011192322), ('wife', 0.9993047714233398), ('dinner', 0.9992475509643555), ('looks', 0.9992450475692749), ('kinds', 0.9992398023605347), ('beef', 0.9991780519485474), ('cherry', 0.9991456270217896), ('blends', 0.9991450309753418), ('eaten', 0.999132513999939)]


In [75]:
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:20])

number of words that occured minimum 5 times  3816
sample words  ['product', 'available', 'course', 'total', 'pretty', 'stinky', 'right', 'nearby', 'used', 'beat', 'great', 'received', 'shipment', 'could', 'hardly', 'wait', 'try', 'love', 'call', 'instead']


In [76]:
#sample vector of a word 
w2v_model.wv.get_vector('great')

array([-0.20346636,  0.05142558, -0.07987118,  0.7056635 ,  0.13991535,
       -0.58735245,  0.4888457 ,  0.11608417,  0.09917501, -0.04789089,
        0.31361514, -0.20386474, -0.8633036 , -0.28058898,  0.1339536 ,
       -0.40925926, -0.64469117, -0.60524803,  0.13972339,  0.8648939 ,
        0.3203178 , -0.11666155,  0.08422046, -0.05454169,  0.6045814 ,
        0.09204479, -0.38339922, -0.34347633,  0.15345779, -0.33865762,
       -0.01031807, -0.66409236,  0.11144474, -0.1375475 , -0.12502065,
        0.59884524, -0.3973102 , -0.21236809, -0.61564946,  0.22381188,
       -0.7147021 , -0.5384901 ,  0.3423769 , -0.60778314, -0.6377711 ,
       -0.06028188,  0.31933197,  0.37180412, -0.92168146, -0.6415185 ],
      dtype=float32)

# Avg Word2Vec - continuation of w2v (w2v model is needed before this)

In [80]:
# Instead of computing vector for each word how to compute one vector for each sentence/Review?? 
#Sum all the vectors of the words present in review, divide by the total number of words present in that review. AvgW2Vec
# list_of_sentance contains list of reviews and each review split on space to have words for that review

# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

100%|██████████| 4986/4986 [00:04<00:00, 1028.85it/s]

4986
50


TFIDF weighted W2v

In [81]:
# S = ["abc def pqr", "def def def abc", "pqr pqr def"]
model = TfidfVectorizer()
model.fit(preprocessed_reviews)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [82]:
# TF-IDF weighted Word2Vec
tfidf_feat = model.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in tqdm(list_of_sentance): # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
#             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

100%|██████████| 4986/4986 [00:36<00:00, 136.05it/s]
